In [1]:
import json
import os
from glob import glob
from tqdm import tqdm
from collections import defaultdict
import pickle
import argparse
import random

In [2]:
indir = os.path.join('data','cofea_processed')
outdir = os.path.join('data','cofea_processed')
data_file = os.path.join(indir, 'cofea.jsonlist')
target_index_file = os.path.join(indir,'target_word_index.dict')
filter_source = ['Evans Early American Imprints','HeinOnline','National Archives Founders Online']
filter_year = [1750,1810]
sample_size = 5000
random.seed(42)

In [3]:
with open(data_file) as f:
     cofea_data = f.readlines()

In [4]:
with open(target_index_file,'rb') as f:
    target_index = pickle.load(f)

In [5]:
# need to get a list of all docs that are in specified sources
elligible_docs = []
for x,doc in enumerate(cofea_data):
    doc = json.loads(doc)
    if doc['source'] in filter_source and (filter_year[0] <= doc['decade'] <= filter_year[1] ):
        elligible_docs.append(x)

elligible_docs = set(elligible_docs)

In [6]:
sampled_target = {}
for word in tqdm(target_index):
    index = []
    for f_ind,doc_ind,ind in target_index[word]:
        # only sample documents from the sources we want and not the first two words in the document
        if doc_ind in elligible_docs and ind > 2:
            index.append((f_ind,doc_ind,ind))
            
    if len(index) > sample_size:
        sample_index = random.sample(index,sample_size)
    else:
        sample_index = index
        
    sampled_target[word] = sample_index

100%|██████████| 1052/1052 [00:18<00:00, 57.13it/s]


In [17]:
with open(os.path.join(outdir,'sample_target_index.dict'),'wb') as f:
    pickle.dump(sampled_target,file=f)

bear arms
301
[(0, 141590, 517), (0, 170408, 11312), (0, 29269, 7529), (0, 172853, 123986), (0, 169807, 58411), (0, 170285, 32780), (0, 172386, 24672), (0, 174003, 39253), (0, 59082, 689), (0, 173439, 5730), (0, 116922, 403), (0, 173448, 168249), (0, 132839, 119), (0, 171862, 157146), (0, 171333, 19757), (0, 170989, 2048), (0, 173921, 107351), (0, 173101, 4651), (0, 172229, 385963), (0, 170840, 501523), (0, 171337, 68372), (0, 171578, 44501), (0, 165827, 919), (0, 171333, 21046), (0, 41408, 721), (0, 174247, 2070), (0, 171333, 20511), (0, 172447, 7258), (0, 171852, 4735), (0, 173281, 47480), (0, 173390, 1096), (0, 172982, 776), (0, 172375, 35494), (0, 172495, 22331), (0, 172994, 5171), (0, 147796, 1216), (0, 121148, 129), (0, 173282, 47840), (0, 172592, 3070), (0, 170378, 4970), (0, 137252, 201), (0, 170840, 228143), (0, 59148, 57), (0, 133832, 179), (0, 172274, 26559), (0, 173925, 142360), (0, 39708, 68), (0, 57404, 949), (0, 65355, 89), (0, 24950, 248), (0, 172418, 10487), (0, 83050,